In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
from shapely.geometry import Point
def intpt_func(row):
    return Point(row['INTPTLON'], row['INTPTLAT'])

In [3]:
#loading geometry data
ham_cbg = pd.read_csv('./ham_cbg.csv')
ham_cbg['intpt'] = ham_cbg[['INTPTLAT', 'INTPTLON']].apply(lambda p: intpt_func(p), axis=1)
ham_cbg = gpd.GeoDataFrame(ham_cbg, geometry=gpd.GeoSeries.from_wkt(ham_cbg.geometry))
ham_cbg.GEOID = ham_cbg.GEOID.astype(str)

c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [4]:
#loading residential buildings
res_build = pd.read_csv('./ham_residential_buildings2.csv', index_col=0)
res_build = gpd.GeoDataFrame(res_build, geometry=gpd.GeoSeries.from_wkt(res_build.geometry))
res_build['location'] = res_build.geometry.apply(lambda p: [p.y, p.x])

#loading work buildings
com_build = pd.read_csv('./work_loc_poi_com_civ.csv', index_col=0)
com_build = gpd.GeoDataFrame(com_build, geometry=gpd.GeoSeries.from_wkt(com_build.geometry))
com_build['location'] = com_build.geometry.apply(lambda p: [p.y, p.x])
com_build = com_build.reset_index()
com_build.GEOID = com_build.GEOID.astype(str)

#loading all buildings (MS dataset)
ms_build = pd.read_csv('./ham_buildings_MS.csv')
ms_build = gpd.GeoDataFrame(ms_build, geometry=gpd.GeoSeries.from_wkt(ms_build.geo_centers))
ms_build.GEOID = ms_build.GEOID.astype(str)
ms_build['location'] = ms_build.geometry.apply(lambda p: [p.y, p.x])

In [5]:
def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta

In [25]:
#generating array of start and return times (in 15 min intervals)
from datetime  import datetime, timedelta
times_morning = [datetime.strptime(dt.strftime('%H:%M'), '%H:%M') for dt in 
       datetime_range(datetime(2016, 9, 1, 7), datetime(2016, 9, 1, 9, 10), 
       timedelta(minutes=15))]
times_evening = [datetime.strptime(dt.strftime('%H:%M'), '%H:%M') for dt in 
       datetime_range(datetime(2016, 9, 1, 16), datetime(2016, 9, 1, 18, 10), 
       timedelta(minutes=15))]

In [7]:
res_build.GEOID = res_build.GEOID.astype(str)
com_build.GEOID = com_build.GEOID.astype(str)

In [8]:
sg = gpd.read_file('../Downloads/ham_sg_jan_mar_21_reduced_cols.csv')


In [13]:
import math
sg.home_cbg = sg.home_cbg.apply(lambda x: str(math.floor(float(x))))
sg.poi_cbg = sg.poi_cbg.apply(lambda x: str(math.floor(float(x))))
sg.frequency = sg.frequency.apply(lambda x: (math.floor(float(x))))
sg.visits_monday = sg.visits_monday.astype(float) 
sg.visits_tuesday = sg.visits_tuesday.astype(float) 
sg.visits_wednesday = sg.visits_wednesday.astype(float) 
sg.visits_thursday = sg.visits_thursday.astype(float) 
sg.visits_friday = sg.visits_friday.astype(float) 
sg.visits_saturday = sg.visits_saturday.astype(float) 
sg.visits_sunday = sg.visits_sunday.astype(float) 

In [14]:
sg = sg.groupby(['home_cbg', 'poi_cbg']).agg(frequency=('frequency', sum), visits_monday=('visits_monday', sum), visits_tuesday=('visits_tuesday', sum), visits_wednesday=('visits_wednesday', sum), visits_thursday=('visits_thursday', sum), visits_friday=('visits_friday', sum), visits_saturday=('visits_saturday', sum), visits_sunday=('visits_sunday', sum) ).reset_index()

In [15]:
pd.set_option('display.max_columns', None)
sg.head()

,home_cbg,poi_cbg,frequency,visits_monday,visits_tuesday,visits_wednesday,visits_thursday,visits_friday,visits_saturday,visits_sunday
0,470650004001,470650004001,601,9152.0,10525.0,10742.0,10662.0,9263.0,3826.0,3736.0
1,470650004001,470650004002,12,27.0,32.0,36.0,18.0,23.0,12.0,10.0
2,470650004001,470650006001,50,467.0,393.0,403.0,415.0,520.0,483.0,557.0
3,470650004001,470650006002,4,10.0,10.0,13.0,7.0,4.0,8.0,6.0
4,470650004001,470650006003,52,738.0,851.0,665.0,875.0,792.0,363.0,214.0


In [30]:
import random
import tqdm
from tqdm.notebook import tqdm_notebook

#setting the random seed
np.random.seed(42)
random.seed(42)


prob_matrix_sg = gpd.GeoDataFrame()
for idx, movement in tqdm_notebook(sg.iterrows(), total=sg.shape[0]):

    res = res_build[res_build.GEOID == movement.home_cbg].reset_index(drop=True)
    if res.empty:
        res = ms_build[ms_build.GEOID == movement.home_cbg].sample(n=movement.frequency, random_state=42, replace=True).reset_index(drop=True)

    com = com_build[com_build.GEOID == movement.poi_cbg].reset_index(drop=True)
    if com.empty:
        com = ms_build[ms_build.GEOID == movement.poi_cbg].sample(n=movement.frequency, random_state=42, replace=True).reset_index(drop=True)
        
    r = res
    c = com

    for freq in range(int(movement.frequency/7)):

        if c.empty:
            c = com
        if r.empty:
            r = res
        
        rand_r = random.randrange(r.shape[0])
        rand_c = random.randrange(c.shape[0])
        r_df = r.iloc[rand_r]
        c_df = c.iloc[rand_c]
        r = r.drop([rand_r]).reset_index(drop=True)
        c = c.drop([rand_c]).reset_index(drop=True)

        time_slot1 = np.random.choice(times_morning, size=1, replace=True)
        time_slot2 = np.random.choice(times_evening, size=1, replace=True)

        temp = gpd.GeoDataFrame()

        temp.loc[freq, 'home_cbg'] = movement.home_cbg
        temp.loc[freq, 'poi_cbg'] = movement.poi_cbg
        temp.loc[freq, 'frequency'] = movement.frequency
        temp.loc[freq, 'home_loc_lat'] = r_df.location[0]
        temp.loc[freq, 'home_loc_lon'] = r_df.location[1]
        temp.loc[freq, 'work_loc_lat'] = c_df.location[0]
        temp.loc[freq, 'work_loc_lon'] = c_df.location[1]
        temp.loc[freq, 'go_time'] = time_slot1[0]
        temp.loc[freq, 'go_time_str'] = time_slot1[0].strftime('%H:%M')
        temp.loc[freq, 'return_time'] = time_slot2[0]
        temp.loc[freq, 'return_time_str'] = time_slot2[0].strftime('%H:%M')

        # temp.loc[job, 'home_geom'] = Point([r_df.location[1], r_df.location[0]])
        prob_matrix_sg = prob_matrix_sg.append(temp, ignore_index=True)

  0%|          | 0/16561 [00:00<?, ?it/s]

In [31]:
def func_home_pt(row):
    return Point(row.home_loc_lon, row.home_loc_lat)
def func_work_pt(row):
    return Point(row.work_loc_lon, row.work_loc_lat)

In [32]:
prob_matrix_sg['home_geom'] = prob_matrix_sg[['home_loc_lat', 'home_loc_lon']].apply(lambda row: func_home_pt(row), axis=1)
prob_matrix_sg['work_geom'] = prob_matrix_sg[['work_loc_lat', 'work_loc_lon']].apply(lambda row: func_work_pt(row), axis=1)

c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
c:\Users\senr1\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [33]:
prob_matrix_sg.head(10)

,home_cbg,poi_cbg,frequency,home_loc_lat,home_loc_lon,work_loc_lat,work_loc_lon,go_time,go_time_str,return_time,return_time_str,home_geom,work_geom
0,470650004001,470650004001,601.0,35.047993,-85.267978,35.044040,-85.271541,1900-01-01 08:30:00,08:30,1900-01-01 16:45:00,16:45,POINT (-85.26797755 35.04799250000001),POINT (-85.271541 35.04404)
1,470650004001,470650004001,601.0,35.049528,-85.263607,35.050207,-85.295038,1900-01-01 08:45:00,08:45,1900-01-01 17:00:00,17:00,POINT (-85.2636066216199 35.04952844187292),POINT (-85.295038 35.050207)
2,470650004001,470650004001,601.0,35.048746,-85.263222,35.046758,-85.288567,1900-01-01 08:30:00,08:30,1900-01-01 16:30:00,16:30,POINT (-85.26322170108523 35.04874566076718),POINT (-85.288567 35.046758)
3,470650004001,470650004001,601.0,35.048906,-85.263567,35.049340,-85.290205,1900-01-01 08:30:00,08:30,1900-01-01 17:45:00,17:45,POINT (-85.26356714606209 35.04890646879263),POINT (-85.290205 35.04934)
4,470650004001,470650004001,601.0,35.047754,-85.268081,35.046745,-85.286959,1900-01-01 08:00:00,08:00,1900-01-01 16:45:00,16:45,POINT (-85.2680808325842 35.04775424922608),POINT (-85.286959 35.046745)
5,470650004001,470650004001,601.0,35.048085,-85.267872,35.045935,-85.279453,1900-01-01 08:45:00,08:45,1900-01-01 17:45:00,17:45,POINT (-85.26787164999999 35.048085300000004),POINT (-85.279453 35.045935)
6,470650004001,470650004001,601.0,35.049181,-85.264166,35.048659,-85.289930,1900-01-01 07:30:00,07:30,1900-01-01 17:15:00,17:15,POINT (-85.26416580000003 35.04918095),POINT (-85.28993 35.048659)
7,470650004001,470650004001,601.0,35.048966,-85.263139,35.050563,-85.290914,1900-01-01 08:00:00,08:00,1900-01-01 16:15:00,16:15,POINT (-85.26313870155471 35.04896578224812),POINT (-85.290914 35.050563)
8,470650004001,470650004001,601.0,35.047867,-85.268032,35.048659,-85.289930,1900-01-01 08:45:00,08:45,1900-01-01 17:15:00,17:15,POINT (-85.26803186621332 35.04786674952997),POINT (-85.28993 35.048659)
9,470650004001,470650004001,601.0,35.047641,-85.268140,35.048697,-85.289601,1900-01-01 07:15:00,07:15,1900-01-01 17:00:00,17:00,POINT (-85.26814047299715 35.04764121644031),POINT (-85.289601 35.048697)


In [34]:
prob_matrix_sg.to_csv('sg_combinations.csv', index=False)